In [1]:
import numpy as np
import cv2 as cv

In [2]:
def read_image():
    image_name = input("Enter the name of the file (with extension)")
    image = cv.imread(image_name)
    print("The shape of the image is = ", image.shape)
    resized_image = cv.resize(image, (500,500))
    cv.imshow("Secret Image",resized_image)
    cv.waitKey(0)

read_image()

The shape of the image is =  (1225, 840, 3)


In [3]:
# Function to convert the secret image into encrypted image
def convertToEncryptedImage(image):
    print("Converting to Encrypted Image...")
    encrypted_image = np.zeros(image.shape)
    print(encrypted_image.shape)